In [8]:
import numpy as np
import pandas as pd

# === Load the dataset ===
data = pd.read_csv("xy_data.csv")
x_obs = data["x"].to_numpy()
y_obs = data["y"].to_numpy()


In [ ]:
#           VALUES IN THE EXAMPLE

# === Parameter values from the example (now in DEGREES) ===
theta = 47.33       # degrees (previously 0.826 radians)
M = 0.0742
X = 11.5793

# === Generate t values (uniformly spaced between 6 and 60) ===
t = np.linspace(6, 60, len(x_obs))

# === Define the model functions (convert degrees -> radians inside) ===
def x_model(t, theta_deg, M, X):
    theta = np.deg2rad(theta_deg)  # convert to radians for math
    return t*np.cos(theta) - np.exp(M*np.abs(t))*np.sin(0.3*t)*np.sin(theta) + X

def y_model(t, theta_deg, M):
    theta = np.deg2rad(theta_deg)  # convert to radians for math
    return 42 + t*np.sin(theta) + np.exp(M*np.abs(t))*np.sin(0.3*t)*np.cos(theta)

# === Predicted values ===
x_pred = x_model(t, theta, M, X)
y_pred = y_model(t, theta, M)

# === Compute L1 distance ===
L1 = np.sum(np.abs(x_obs - x_pred) + np.abs(y_obs - y_pred))

print(f"Theta = {theta:.3f} degrees")
print(f"L1 distance = {L1:.6f}")


Theta = 47.330 degrees
L1 distance = 96168.173333


In [ ]:
#           ITERATING EACH INDIVIUAL VARIABLE


# --- Define the parametric model (from the question) ---
def curve(t, theta, M, X):
    """Return predicted x(t), y(t) for given params."""
    x_pred = (t * np.cos(theta)
              - np.exp(M * np.abs(t)) * np.sin(0.3 * t) * np.sin(theta)
              + X)
    y_pred = (42
              + t * np.sin(theta)
              + np.exp(M * np.abs(t)) * np.sin(0.3 * t) * np.cos(theta))
    return x_pred, y_pred


# --- Define L1 Loss Function ---
def L1_loss(params):
    """
    Compute the L1 (Manhattan) distance between predicted and observed curves.
    
    Formula:
        L1 = mean( |x_pred - x_obs| + |y_pred - y_obs| )

    Lower L1 => better fit.
    """
    theta, M, X = params
    x_pred, y_pred = curve(t, theta, M, X)
    return np.sum(np.abs(x_pred - x_obs) + np.abs(y_pred - y_obs))


# --- Coordinate Descent Optimization ---
def coordinate_descent(theta0, M0, X0, iters=5):
    theta, M, X = theta0, M0, X0
    loss_history = []

    for i in range(iters):
        # 1. Optimize theta
        theta_grid = np.linspace(theta - np.deg2rad(1), theta + np.deg2rad(1), 30)
        theta = min(theta_grid, key=lambda th: L1_loss((th, M, X)))

        # 2. Optimize M
        M_grid = np.linspace(M - 0.005, M + 0.005, 30)
        M = min(M_grid, key=lambda m: L1_loss((theta, m, X)))

        # 3. Optimize X
        X_grid = np.linspace(X - 2, X + 2, 30)
        X = min(X_grid, key=lambda x_: L1_loss((theta, M, x_)))

        current_loss = L1_loss((theta, M, X))
        loss_history.append(current_loss)

        print(f"Iter {i+1}: θ={np.rad2deg(theta):.3f}°, M={M:.5f}, X={X:.3f}, L1={current_loss:.6f}")

    return theta, M, X, loss_history


# --- Initial Guesses (within given range) ---
theta0 = np.deg2rad(25)  # mid of 0–50 degrees
M0 = 0.0                 # mid of -0.05–0.05
X0 = 50.0                # mid of 0–100

# --- Run the Optimization ---
theta_opt, M_opt, X_opt, loss_history = coordinate_descent(theta0, M0, X0, iters=1000)

# --- Print Results ---
print("\n--- Final Optimized Values ---")
print(f"Theta = {np.rad2deg(theta_opt):.4f} degrees")
print(f"M     = {M_opt:.6f}")
print(f"X     = {X_opt:.4f}")
print(f"Final L1 Loss = {loss_history[-1]:.6f}")


Iter 1: θ=25.793°, M=0.00500, X=52.000, L1=38117.892738
Iter 2: θ=26.517°, M=0.01000, X=54.000, L1=37930.995882
Iter 3: θ=27.448°, M=0.01500, X=54.759, L1=37880.264873
Iter 4: θ=27.759°, M=0.02000, X=54.828, L1=37868.050453
Iter 5: θ=28.000°, M=0.02086, X=54.897, L1=37865.541961
Iter 6: θ=28.103°, M=0.02138, X=54.828, L1=37865.332365
Iter 7: θ=28.138°, M=0.02155, X=54.897, L1=37865.142698
Iter 8: θ=28.103°, M=0.02138, X=54.828, L1=37865.332365
Iter 9: θ=28.138°, M=0.02155, X=54.897, L1=37865.142698
Iter 10: θ=28.103°, M=0.02138, X=54.828, L1=37865.332365
Iter 11: θ=28.138°, M=0.02155, X=54.897, L1=37865.142698
Iter 12: θ=28.103°, M=0.02138, X=54.828, L1=37865.332365
Iter 13: θ=28.138°, M=0.02155, X=54.897, L1=37865.142698
Iter 14: θ=28.103°, M=0.02138, X=54.828, L1=37865.332365
Iter 15: θ=28.138°, M=0.02155, X=54.897, L1=37865.142698
Iter 16: θ=28.103°, M=0.02138, X=54.828, L1=37865.332365
Iter 17: θ=28.138°, M=0.02155, X=54.897, L1=37865.142698
Iter 18: θ=28.103°, M=0.02138, X=54.828,